Imports

In [ ]:
import cv2
import numpy as np
import os
from time import time
import random
import pandas as pd
import math

from random import seed
from random import randrange

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from scipy.cluster.vq import kmeans, vq
from sklearn.metrics import confusion_matrix, classification_report,precision_score,f1_score,recall_score,accuracy_score
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

Face and eyes detection functions

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image,minNeighbors=6,minSize=(75, 75)) #oi parametroi allazoun analoga to dataset
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        
        return False, "zero"
    else:  #if more than 1 faces are detected, the biggest is kept (the smaller ones usually belong in the background)
        biggest=0
        pos=0
        for i,(x,y,w,h) in enumerate(face):
            if biggest<(h*w):
                biggest=h*w
                pos=i
        return True, image[y:y+w, x:x+h], face[pos]

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image)#,scaleFactor=1.2, minNeighbors=5)
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        '''
        for (x,y,w,h) in face: 
            cv2.rectangle(image, (x,y), (x+w,y+h), (255,255,255), 3)
        plt.imshow(image)
        plt.show()
        '''
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        return False, "zero"
    else: #if more than 1 faces are detected
        return False, "more"

In [ ]:
def eye_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye.xml')
    eyes = haar_classifier.detectMultiScale(image, minNeighbors=4, minSize=(30, 30))
    if (len(eyes)==2): #if 2 eyes are detected
        return True, eyes
    else:
        haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye_tree_eyeglasses.xml')
        glasses = haar_classifier.detectMultiScale(image)
        if (len(glasses)==2):
            return True, glasses
        else:
            return False, "more or less"

In [ ]:
train_path = 'C:/Users/Thomas/Documents/Datasets/ExtYale(640x480)/'  
training_names = os.listdir(train_path)
min_faces=30
scores=[]
target_names=[]  #all the names are saved here

faces=[]
labels=[]

total_photos_seen=0 
n_classes=0
folders = os.listdir(train_path)
for folder in folders:
    label = os.path.basename(folder)
    training_images_path = train_path + '/' + folder
    num_of_faces = len(os.listdir(training_images_path))
    #if num_of_faces>=min_faces:   #people with low number of faces are skipped
    target_names.append(label)
    n_classes=n_classes+1
    faces_per_person=0
    for image in os.listdir(training_images_path):
        total_photos_seen=total_photos_seen+1
        image_path = training_images_path + '/' + image
        training_image = cv2.imread(image_path)
        #face=cv2.cvtColor(training_image,cv2.COLOR_BGR2GRAY)
        #resized=cv2.resize(face,(224,224),interpolation=cv2.INTER_AREA)
        result_im=face_detection(training_image) #this function returns TRUE/FALSE, the cut image and the coordinates of where the cut was made
        if result_im[0]: #if TRUE (face detected) then the cut face goes into the array
            cut_face=result_im[1]
            faces.append(cut_face)
            labels.append(n_classes)

        '''
        if faces_per_person==45:
                break
        '''       

In [ ]:
new_faces=[]
new_labels=[]

In [ ]:
rotated_faces=0
for i in range(len(faces)):
    training_image = cv2.imread(faces[i])
    result_im=eye_detection(training_image) #finding the place of the eyes
    
    if result_im[0]: #process of finding the angle of the two eyes, and alighnment
        rotated_faces=rotated_faces+1
        (x1,y1,w1,h1)=result_im[1][0]
        (x2,y2,w2,h2)=result_im[1][1]
        eye1_center_x=x1+(w1/2)
        eye1_center_y=y1+(h1/2)
        eye2_center_x=x2+(w2/2)
        eye2_center_y=y2+(h2/2)
        if (eye1_center_x>eye2_center_y):
            deltaY = eye1_center_y - eye2_center_y 
            deltaX = eye1_center_x - eye2_center_x
        else:
            deltaY = eye2_center_y - eye1_center_y 
            deltaX = eye2_center_x - eye1_center_x
        my_radians = math.atan2(deltaY , deltaX)
        my_degrees = math.degrees(my_radians)

        #rotation
        rows, cols = training_image.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), my_degrees, 1)
        img_rotated = cv2.warpAffine(training_image, M, (cols,rows))
        '''
        font = cv2.FONT_HERSHEY_SIMPLEX
        plt.text(0,0,i)
        plt.imshow(img_rotated)
        plt.show()
        '''

        #scaling (all images will become 200x200)
        #if before the conversion the image is smaller than 200x200 INTER_LINEAR will be used, else INTER_AREA will be used
        image_size=rows*cols        
        if (image_size<=40000):
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_LINEAR)
        else:
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_AREA)

        #normalization
        norm_img = np.zeros((200, 200))
        norm_img = cv2.normalize(scaled_image, norm_img, 0, 255, cv2.NORM_MINMAX)

        final_image = cv2.cvtColor(norm_img, cv2.COLOR_BGR2GRAY)

        new_faces.append(final_image)
        new_labels.append(labels[i])
        
faces=new_faces
labels=new_labels

In [ ]:
orb = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
brisk = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
sift = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
surf = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})

In [ ]:
orb_learning_curve = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
brisk_learning_curve = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
sift_learning_curve = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})
surf_learning_curve = dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})

ORB

In [ ]:
train_sizes1=np.empty(5)
train_sizes1[:]=np.NaN
train_scores1=np.empty([5, 10])
train_scores1[:]=np.NaN
test_scores1=np.empty([5, 10])
test_scores1[:]=np.NaN
fit_times1=np.empty([5, 10])
fit_times1[:]=np.NaN
score_times1=np.empty([5, 10])
score_times1[:]=np.NaN
cluster_times1=np.empty([5, 10])
cluster_times1[:]=np.NaN

train_sizes2=np.empty(5)
train_sizes2[:]=np.NaN
train_scores2=np.empty([5, 10])
train_scores2[:]=np.NaN
test_scores2=np.empty([5, 10])
test_scores2[:]=np.NaN
fit_times2=np.empty([5, 10])
fit_times2[:]=np.NaN
score_times2=np.empty([5, 10])
score_times2[:]=np.NaN
cluster_times2=np.empty([5, 10])
cluster_times2[:]=np.NaN

train_sizes3=np.empty(5)
train_sizes3[:]=np.NaN
train_scores3=np.empty([5, 10])
train_scores3[:]=np.NaN
test_scores3=np.empty([5, 10])
test_scores3[:]=np.NaN
fit_times3=np.empty([5, 10])
fit_times3[:]=np.NaN
score_times3=np.empty([5, 10])
score_times3[:]=np.NaN
cluster_times3=np.empty([5, 10])
cluster_times3[:]=np.NaN

train_sizes4=np.empty(5)
train_sizes4[:]=np.NaN
train_scores4=np.empty([5, 10])
train_scores4[:]=np.NaN
test_scores4=np.empty([5, 10])
test_scores4[:]=np.NaN
fit_times4=np.empty([5, 10])
fit_times4[:]=np.NaN
score_times4=np.empty([5, 10])
score_times4[:]=np.NaN
cluster_times4=np.empty([5, 10])
cluster_times4[:]=np.NaN

train_sizes5=np.empty(5)
train_sizes5[:]=np.NaN
train_scores5=np.empty([5, 10])
train_scores5[:]=np.NaN
test_scores5=np.empty([5, 10])
test_scores5[:]=np.NaN
fit_times5=np.empty([5, 10])
fit_times5[:]=np.NaN
score_times5=np.empty([5, 10])
score_times5[:]=np.NaN
cluster_times5=np.empty([5, 10])
cluster_times5[:]=np.NaN

train_sizes6=np.empty(5)
train_sizes6[:]=np.NaN
train_scores6=np.empty([5, 10])
train_scores6[:]=np.NaN
test_scores6=np.empty([5, 10])
test_scores6[:]=np.NaN
fit_times6=np.empty([5, 10])
fit_times6[:]=np.NaN
score_times6=np.empty([5, 10])
score_times6[:]=np.NaN
cluster_times6=np.empty([5, 10])
cluster_times6[:]=np.NaN

train_sizes7=np.empty(5)
train_sizes7[:]=np.NaN
train_scores7=np.empty([5, 10])
train_scores7[:]=np.NaN
test_scores7=np.empty([5, 10])
test_scores7[:]=np.NaN
fit_times7=np.empty([5, 10])
fit_times7[:]=np.NaN
score_times7=np.empty([5, 10])
score_times7[:]=np.NaN
cluster_times7=np.empty([5, 10])
cluster_times7[:]=np.NaN

train_sizes8=np.empty(5)
train_sizes8[:]=np.NaN
train_scores8=np.empty([5, 10])
train_scores8[:]=np.NaN
test_scores8=np.empty([5, 10])
test_scores8[:]=np.NaN
fit_times8=np.empty([5, 10])
fit_times8[:]=np.NaN
score_times8=np.empty([5, 10])
score_times8[:]=np.NaN
cluster_times8=np.empty([5, 10])
cluster_times8[:]=np.NaN

train_sizes9=np.empty(5)
train_sizes9[:]=np.NaN
train_scores9=np.empty([5, 10])
train_scores9[:]=np.NaN
test_scores9=np.empty([5, 10])
test_scores9[:]=np.NaN
fit_times9=np.empty([5, 10])
fit_times9[:]=np.NaN
score_times9=np.empty([5, 10])
score_times9[:]=np.NaN
cluster_times9=np.empty([5, 10])
cluster_times9[:]=np.NaN

train_sizes10=np.empty(5)
train_sizes10[:]=np.NaN
train_scores10=np.empty([5, 5])
train_scores10[:]=np.NaN
test_scores10=np.empty([5, 5])
test_scores10[:]=np.NaN
fit_times10=np.empty([5, 5])
fit_times10[:]=np.NaN
score_times10=np.empty([5, 5])
score_times10[:]=np.NaN
cluster_times10=np.empty([5, 5])
cluster_times10[:]=np.NaN

train_sizes11=np.empty(5)
train_sizes11[:]=np.NaN
train_scores11=np.empty([5, 10])
train_scores11[:]=np.NaN
test_scores11=np.empty([5, 10])
test_scores11[:]=np.NaN
fit_times11=np.empty([5, 10])
fit_times11[:]=np.NaN
score_times11=np.empty([5, 10])
score_times11[:]=np.NaN
cluster_times11=np.empty([5, 10])
cluster_times11[:]=np.NaN

In [ ]:
svc_learning_curve=dict({1:train_sizes1,2:train_scores1,3:test_scores1,4:fit_times1,5:score_times1,6:cluster_times1})
mlp_learning_curve=dict({1:train_sizes2,2:train_scores2,3:test_scores2,4:fit_times2,5:score_times2,6:cluster_times2})
knn_learning_curve=dict({1:train_sizes3,2:train_scores3,3:test_scores3,4:fit_times3,5:score_times3,6:cluster_times3})
logreg_learning_curve=dict({1:train_sizes4,2:train_scores4,3:test_scores4,4:fit_times4,5:score_times4,6:cluster_times4})
linsvc_learning_curve=dict({1:train_sizes5,2:train_scores5,3:test_scores5,4:fit_times5,5:score_times5,6:cluster_times5})
tree_learning_curve=dict({1:train_sizes6,2:train_scores6,3:test_scores6,4:fit_times6,5:score_times6,6:cluster_times6})
forrest_learning_curve=dict({1:train_sizes7,2:train_scores7,3:test_scores7,4:fit_times7,5:score_times7,6:cluster_times7})
adaboost_learning_curve=dict({1:train_sizes8,2:train_scores8,3:test_scores8,4:fit_times8,5:score_times8,6:cluster_times8})
gnb_learning_curve=dict({1:train_sizes9,2:train_scores9,3:test_scores9,4:fit_times9,5:score_times9,6:cluster_times9})
gpc_learning_curve=dict({1:train_sizes10,2:train_scores10,3:test_scores10,4:fit_times10,5:score_times10,6:cluster_times10})
ridge_learning_curve=dict({1:train_sizes11,2:train_scores11,3:test_scores11,4:fit_times11,5:score_times11,6:cluster_times11})

In [ ]:
names=["accuracy","macro_avg_precision","macro_avg_recall","macro_avg_f1_score",
       "micro_avg_precision","micro_avg_recall","micro_avg_f1_score",
       "weighted_avg_precision","weighted_avg_recall","weighted_avg_f1_score",
       "fit_time","cluster_time"]

df_svc = pd.DataFrame(index=names)
df_mlp = pd.DataFrame(index=names)
df_knn = pd.DataFrame(index=names)
df_logreg = pd.DataFrame(index=names)
df_linsvc = pd.DataFrame(index=names)
df_tree = pd.DataFrame(index=names)
df_forrest = pd.DataFrame(index=names)
df_adaboost = pd.DataFrame(index=names)
df_gnb = pd.DataFrame(index=names)
df_gpc = pd.DataFrame(index=names)
df_ridge = pd.DataFrame(index=names)


faces = np.array(faces)
labels=np.array(labels)

des_list = []

orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=train_path
    
    kpts, des = orb.detectAndCompute(face, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    #kpts, des = surf.detectAndCompute(im, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        print(image_path)
        list_to_delete.append(i)

new_faces_ = [j for i, j in enumerate(faces) if i not in list_to_delete]
faces=new_faces_

new_labels_ = [j for i, j in enumerate(labels) if i not in list_to_delete]
labels=new_labels_
print("done in %0.3fs" % (time() - t0))
print("Found descriptors for ",len(des_list)," images")
print("Found  ",len(labels)," labels")

labels=np.array(labels)
des_list=np.array(des_list)


In [ ]:
kf=StratifiedKFold(n_splits=10)

In [ ]:
for size in range(1,6):
    print("--------------------------Start for size: ",size,"----------------------------------------")
    for k_f,idx in enumerate(kf.split(X=des_list, y=labels)):
        train_idx,test_idx=idx[0],idx[1]
        x_train=des_list[train_idx]
        y_train=labels[train_idx]

        x_test=des_list[test_idx]
        y_test=labels[test_idx]

        print("Fold Number : ",k_f+1)

        #Training==================================================================================================================================    
        if (size==1):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.22,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.22,shuffle=True,stratify=y_test)
        elif (size==2):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.325,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.325,shuffle=True,stratify=y_test)
        elif (size==3):
            X_train_new, X, y_train_new, y = train_test_split(x_train, y_train, train_size=0.55,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.55,shuffle=True,stratify=y_test)
        elif (size==4):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.77,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.77,shuffle=True,stratify=y_test)
        else:
            X_train_new=x_train
            y_train_new=y_train
            X_test_new=x_test
            y_test_new=y_test
        
        svc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        mlp_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        knn_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        logreg_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        linsvc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        tree_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        forrest_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        adaboost_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        gnb_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        if (k_f<5):
            gpc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        ridge_learning_curve[1][size-1]=len(X_train_new)#setting train_size

        print("Stacking...")
        t0 = time()
        descriptors = X_train_new[0][1]
        for image_path, descriptor in X_train_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        descriptors_float = descriptors.astype(float) 

        print("Creating clusters and histogram...")
        t0 = time()
        k = 200
        voc, variance = kmeans(descriptors_float, k, 1)

        im_features = np.zeros((len(X_train_new), k), "float32")
        for i in range(len(X_train_new)):
            words, distance = vq(X_train_new[i][1],voc)
            for w in words:
                im_features[i][w] += 1

        nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_train_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        stdSlr = StandardScaler().fit(im_features)
        im_features = stdSlr.transform(im_features)
        cluster_time=time() - t0
        print("done in %0.3fs" %cluster_time)
        svc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        mlp_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        knn_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        logreg_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        linsvc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        tree_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        forrest_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        adaboost_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        gnb_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        if (k_f<5):
            gpc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        ridge_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time


        #Here comes the fitting of the model
        print("Training the svc model...")############################################################################
        t0 = time()

        svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
        svc.fit(im_features, np.array(y_train_new))

        svc_fit_time=time() - t0
        print("Training the svc done in %0.3fs" %svc_fit_time)
        svc_learning_curve[4][size-1][k_f]=svc_fit_time#setting fit_time
        test_sc=svc.score(im_features, np.array(y_train_new))
        svc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        
        print("Training the linsvc model...")###################################################################
        t0 = time()

        linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
        linsvc.fit(im_features, np.array(y_train_new))

        linsvc_fit_time=time() - t0
        print("Training the linsvc done in %0.3fs" %linsvc_fit_time)
        linsvc_learning_curve[4][size-1][k_f]=linsvc_fit_time#setting fit_time
        test_sc=linsvc.score(im_features, np.array(y_train_new))
        linsvc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the forrest model...")#####################################################################
        t0 = time()

        forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
        forrest.fit(im_features, np.array(y_train_new))

        forrest_fit_time=time() - t0
        print("Training the forrest done in %0.3fs" %forrest_fit_time)
        forrest_learning_curve[4][size-1][k_f]=forrest_fit_time#setting fit_time
        test_sc=forrest.score(im_features, np.array(y_train_new))
        forrest_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the logreg model...")#######################################################################
        t0 = time()

        logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
        logreg.fit(im_features, np.array(y_train_new))

        logreg_fit_time=time() - t0
        print("Training the logreg done in %0.3fs" %logreg_fit_time)
        logreg_learning_curve[4][size-1][k_f]=logreg_fit_time#setting fit_time
        test_sc=logreg.score(im_features, np.array(y_train_new))
        logreg_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the tree model...")#######################################################################
        t0 = time()

        tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                                  max_leaf_nodes=7,max_features=None)    
        tree.fit(im_features, np.array(y_train_new))

        tree_fit_time=time() - t0
        print("Training the tree done in %0.3fs" %tree_fit_time)
        tree_learning_curve[4][size-1][k_f]=tree_fit_time#setting fit_time
        test_sc=tree.score(im_features, np.array(y_train_new))
        tree_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the knn model...")#########################################################################
        t0 = time()

        knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
        knn.fit(im_features, np.array(y_train_new))

        knn_fit_time=time() - t0
        print("Training the knn done in %0.3fs" %knn_fit_time)
        knn_learning_curve[4][size-1][k_f]=knn_fit_time#setting fit_time
        test_sc=knn.score(im_features, np.array(y_train_new))
        knn_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the mlp model...")###########################################################################
        t0 = time()

        mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
        mlp.fit(im_features, np.array(y_train_new))

        mlp_fit_time=time() - t0
        print("Training the mlp done in %0.3fs" %mlp_fit_time)
        mlp_learning_curve[4][size-1][k_f]=mlp_fit_time#setting fit_time
        test_sc=mlp.score(im_features, np.array(y_train_new))
        mlp_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the adaboost model...")########################################################################
        t0 = time()

        adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
        adaboost.fit(im_features, np.array(y_train_new))

        adaboost_fit_time=time() - t0
        print("Training the adaboost done in %0.3fs" %adaboost_fit_time)
        adaboost_learning_curve[4][size-1][k_f]=adaboost_fit_time#setting fit_time
        test_sc=adaboost.score(im_features, np.array(y_train_new))
        adaboost_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gnb model...")###############################################################
        t0 = time()

        gnb = GaussianNB()    
        gnb.fit(im_features, np.array(y_train_new))

        gnb_fit_time=time() - t0
        print("Training the gnb done in %0.3fs" %gnb_fit_time)
        gnb_learning_curve[4][size-1][k_f]=gnb_fit_time#setting fit_time
        test_sc=gnb.score(im_features, np.array(y_train_new))
        gnb_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        if (k_f<5): #GPC has less training because it takes a lot of time
            print("Training the gpc model...")##############################################################
            t0 = time()

            gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
            gpc.fit(im_features, np.array(y_train_new))

            gpc_fit_time=time() - t0
            print("Training the gpc done in %0.3fs" %gpc_fit_time)
            gpc_learning_curve[4][size-1][k_f]=gpc_fit_time#setting fit_time
            test_sc=gpc.score(im_features, np.array(y_train_new))
            gpc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the ridge model...")############################################################
        t0 = time()

        ridge = RidgeClassifier(alpha=0.5,solver='saga')    
        ridge.fit(im_features, np.array(y_train_new))

        ridge_fit_time=time() - t0
        print("Training the ridge done in %0.3fs" %ridge_fit_time)
        ridge_learning_curve[4][size-1][k_f]=ridge_fit_time#setting fit_time
        test_sc=ridge.score(im_features, np.array(y_train_new))
        ridge_learning_curve[2][size-1][k_f]=test_sc#setting train_score



        #Testing=============================================================================================================================
        print("Testing...")     

        print("Stacking...")
        t0 = time()
        descriptors = X_test_new[0][1]
        for image_path, descriptor in X_test_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        print("Calculating histogram, Scaling...")
        test_features = np.zeros((len(X_test_new), k), "float32")
        for i in range(len(X_test_new)):
            words, distance = vq(X_test_new[i][1],voc)
            for w in words:
                test_features[i][w] += 1

        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_test_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)
        print("done in %0.3fs" % (time() - t0))

        if (size==5):
        
            true_class =  [training_names[i-1] for i in y_test_new]
            print("Predicting Started")
            #svc========================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in svc.predict(test_features)]
            score_t=time() - t0
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            svc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_svc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       svc_fit_time,cluster_time]

            #mlp====================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in mlp.predict(test_features)]
            score_t=time() - t0
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            mlp_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_mlp[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       mlp_fit_time,cluster_time]

            #knn=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in knn.predict(test_features)]
            score_t=time() - t0
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            knn_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_knn[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       knn_fit_time,cluster_time]

            #logreg==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in logreg.predict(test_features)]
            score_t=time() - t0
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            logreg_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_logreg[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       logreg_fit_time,cluster_time]

            #linsvc===================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in linsvc.predict(test_features)]
            score_t=time() - t0
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            linsvc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_linsvc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       linsvc_fit_time,cluster_time]

            #tree=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in tree.predict(test_features)]
            score_t=time() - t0
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            tree_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_tree[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       tree_fit_time,cluster_time]

            #forrest======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in forrest.predict(test_features)]
            score_t=time() - t0
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            forrest_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_forrest[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       forrest_fit_time,cluster_time]

            #adaboost=======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in adaboost.predict(test_features)]
            score_t=time() - t0
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions, output_dict=True)
            accuracy=report['accuracy']
            adaboost_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_adaboost[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       adaboost_fit_time,cluster_time]

            #gnb=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in gnb.predict(test_features)]
            score_t=time() - t0
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            gnb_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_gnb[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       gnb_fit_time,cluster_time]

            #gpc============================================================================================
            if (k_f<5):
                t0 = time()
                predictions =  [training_names[i-1] for i in gpc.predict(test_features)]
                score_t=time() - t0
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time


                report=classification_report(true_class, predictions,output_dict=True)
                accuracy=report['accuracy']
                gpc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
                macro_avg_precision=report['macro avg']['precision']
                macro_avg_recall=report['macro avg']['recall']
                macro_avg_f1_score=report['macro avg']['f1-score']
                micro_avg_precision=precision_score(true_class, predictions, average='micro')
                micro_avg_recall=recall_score(true_class, predictions, average='micro')
                micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
                weighted_avg_precision=report['weighted avg']['precision']
                weighted_avg_recall=report['weighted avg']['recall']
                weighted_avg_f1_score=report['weighted avg']['f1-score']

                df_gpc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                           micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                           weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                           gpc_fit_time,cluster_time]

            #ridge==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in ridge.predict(test_features)]
            score_t=time() - t0
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            ridge_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_ridge[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       ridge_fit_time,cluster_time]

        else:
            #svc=============================
            t0 = time()
            test_scores=svc.score(test_features,y_test_new)
            score_t=time() - t0
            svc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #mlp==================================
            t0 = time()
            test_scores=mlp.score(test_features,y_test_new)
            score_t=time() - t0
            mlp_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #knn=============================
            t0 = time()
            test_scores=knn.score(test_features,y_test_new)
            score_t=time() - t0
            knn_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #logreg====================================
            t0 = time()
            test_scores=logreg.score(test_features,y_test_new)
            score_t=time() - t0
            logreg_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #linsvc=========================================
            t0 = time()
            test_scores=linsvc.score(test_features,y_test_new)
            score_t=time() - t0
            linsvc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #tree======================================
            t0 = time()
            test_scores=tree.score(test_features,y_test_new)
            score_t=time() - t0
            tree_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #forrest=========================================
            t0 = time()
            test_scores=forrest.score(test_features,y_test_new)
            score_t=time() - t0
            forrest_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #adaboost==========================================
            t0 = time()
            test_scores=adaboost.score(test_features,y_test_new)
            score_t=time() - t0
            adaboost_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gnb===========================================
            t0 = time()
            test_scores=gnb.score(test_features,y_test_new)
            score_t=time() - t0
            gnb_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gpc========================================
            if (k_f<5):
                t0 = time()
                test_scores=gpc.score(test_features,y_test_new)
                score_t=time() - t0
                gpc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #ridge=========================================
            t0 = time()
            test_scores=ridge.score(test_features,y_test_new)
            score_t=time() - t0
            ridge_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            
        print("Predicting DONE for fold ",k_f+1)

        
    print("END OF LOOP for size: ",size)
    
print("End of ALL")
lfw['svc']=df_svc
lfw['mlp']=df_mlp
lfw['knn']=df_knn
lfw['logreg']=df_logreg
lfw['linsvc']=df_linsvc
lfw['tree']=df_tree
lfw['forrest']=df_forrest
lfw['adaboost']=df_adaboost
lfw['gnb']=df_gnb
lfw['gpc']=df_gpc
lfw['ridge']=df_ridge


learning_curve_lfw['svc']=svc_learning_curve
learning_curve_lfw['mlp']=mlp_learning_curve
learning_curve_lfw['knn']=knn_learning_curve
learning_curve_lfw['logreg']=logreg_learning_curve
learning_curve_lfw['linsvc']=linsvc_learning_curve
learning_curve_lfw['tree']=tree_learning_curve
learning_curve_lfw['forrest']=forrest_learning_curve
learning_curve_lfw['adaboost']=adaboost_learning_curve
learning_curve_lfw['gnb']=gnb_learning_curve
learning_curve_lfw['gpc']=gpc_learning_curve
learning_curve_lfw['ridge']=ridge_learning_curve

In [ ]:
pickle_out = open('orb_extyale', 'wb')
pickle.dump(orb_lfw, pickle_out)
pickle_out.close()
#pickle_in = open('my_dict.pickle', 'rb')
#new_dict = pickle.load(pickle_in)

In [ ]:
with open('orb_learning_curve_extyale.p', 'wb') as fp:
        pickle.dump(orb_learning_curve_lfw, fp, protocol=pickle.HIGHEST_PROTOCOL)

BRISK

In [ ]:
train_sizes1=np.empty(5)
train_sizes1[:]=np.NaN
train_scores1=np.empty([5, 10])
train_scores1[:]=np.NaN
test_scores1=np.empty([5, 10])
test_scores1[:]=np.NaN
fit_times1=np.empty([5, 10])
fit_times1[:]=np.NaN
score_times1=np.empty([5, 10])
score_times1[:]=np.NaN
cluster_times1=np.empty([5, 10])
cluster_times1[:]=np.NaN

train_sizes2=np.empty(5)
train_sizes2[:]=np.NaN
train_scores2=np.empty([5, 10])
train_scores2[:]=np.NaN
test_scores2=np.empty([5, 10])
test_scores2[:]=np.NaN
fit_times2=np.empty([5, 10])
fit_times2[:]=np.NaN
score_times2=np.empty([5, 10])
score_times2[:]=np.NaN
cluster_times2=np.empty([5, 10])
cluster_times2[:]=np.NaN

train_sizes3=np.empty(5)
train_sizes3[:]=np.NaN
train_scores3=np.empty([5, 10])
train_scores3[:]=np.NaN
test_scores3=np.empty([5, 10])
test_scores3[:]=np.NaN
fit_times3=np.empty([5, 10])
fit_times3[:]=np.NaN
score_times3=np.empty([5, 10])
score_times3[:]=np.NaN
cluster_times3=np.empty([5, 10])
cluster_times3[:]=np.NaN

train_sizes4=np.empty(5)
train_sizes4[:]=np.NaN
train_scores4=np.empty([5, 10])
train_scores4[:]=np.NaN
test_scores4=np.empty([5, 10])
test_scores4[:]=np.NaN
fit_times4=np.empty([5, 10])
fit_times4[:]=np.NaN
score_times4=np.empty([5, 10])
score_times4[:]=np.NaN
cluster_times4=np.empty([5, 10])
cluster_times4[:]=np.NaN

train_sizes5=np.empty(5)
train_sizes5[:]=np.NaN
train_scores5=np.empty([5, 10])
train_scores5[:]=np.NaN
test_scores5=np.empty([5, 10])
test_scores5[:]=np.NaN
fit_times5=np.empty([5, 10])
fit_times5[:]=np.NaN
score_times5=np.empty([5, 10])
score_times5[:]=np.NaN
cluster_times5=np.empty([5, 10])
cluster_times5[:]=np.NaN

train_sizes6=np.empty(5)
train_sizes6[:]=np.NaN
train_scores6=np.empty([5, 10])
train_scores6[:]=np.NaN
test_scores6=np.empty([5, 10])
test_scores6[:]=np.NaN
fit_times6=np.empty([5, 10])
fit_times6[:]=np.NaN
score_times6=np.empty([5, 10])
score_times6[:]=np.NaN
cluster_times6=np.empty([5, 10])
cluster_times6[:]=np.NaN

train_sizes7=np.empty(5)
train_sizes7[:]=np.NaN
train_scores7=np.empty([5, 10])
train_scores7[:]=np.NaN
test_scores7=np.empty([5, 10])
test_scores7[:]=np.NaN
fit_times7=np.empty([5, 10])
fit_times7[:]=np.NaN
score_times7=np.empty([5, 10])
score_times7[:]=np.NaN
cluster_times7=np.empty([5, 10])
cluster_times7[:]=np.NaN

train_sizes8=np.empty(5)
train_sizes8[:]=np.NaN
train_scores8=np.empty([5, 10])
train_scores8[:]=np.NaN
test_scores8=np.empty([5, 10])
test_scores8[:]=np.NaN
fit_times8=np.empty([5, 10])
fit_times8[:]=np.NaN
score_times8=np.empty([5, 10])
score_times8[:]=np.NaN
cluster_times8=np.empty([5, 10])
cluster_times8[:]=np.NaN

train_sizes9=np.empty(5)
train_sizes9[:]=np.NaN
train_scores9=np.empty([5, 10])
train_scores9[:]=np.NaN
test_scores9=np.empty([5, 10])
test_scores9[:]=np.NaN
fit_times9=np.empty([5, 10])
fit_times9[:]=np.NaN
score_times9=np.empty([5, 10])
score_times9[:]=np.NaN
cluster_times9=np.empty([5, 10])
cluster_times9[:]=np.NaN

train_sizes10=np.empty(5)
train_sizes10[:]=np.NaN
train_scores10=np.empty([5, 5])
train_scores10[:]=np.NaN
test_scores10=np.empty([5, 5])
test_scores10[:]=np.NaN
fit_times10=np.empty([5, 5])
fit_times10[:]=np.NaN
score_times10=np.empty([5, 5])
score_times10[:]=np.NaN
cluster_times10=np.empty([5, 5])
cluster_times10[:]=np.NaN

train_sizes11=np.empty(5)
train_sizes11[:]=np.NaN
train_scores11=np.empty([5, 10])
train_scores11[:]=np.NaN
test_scores11=np.empty([5, 10])
test_scores11[:]=np.NaN
fit_times11=np.empty([5, 10])
fit_times11[:]=np.NaN
score_times11=np.empty([5, 10])
score_times11[:]=np.NaN
cluster_times11=np.empty([5, 10])
cluster_times11[:]=np.NaN

In [ ]:
svc_learning_curve=dict({1:train_sizes1,2:train_scores1,3:test_scores1,4:fit_times1,5:score_times1,6:cluster_times1})
mlp_learning_curve=dict({1:train_sizes2,2:train_scores2,3:test_scores2,4:fit_times2,5:score_times2,6:cluster_times2})
knn_learning_curve=dict({1:train_sizes3,2:train_scores3,3:test_scores3,4:fit_times3,5:score_times3,6:cluster_times3})
logreg_learning_curve=dict({1:train_sizes4,2:train_scores4,3:test_scores4,4:fit_times4,5:score_times4,6:cluster_times4})
linsvc_learning_curve=dict({1:train_sizes5,2:train_scores5,3:test_scores5,4:fit_times5,5:score_times5,6:cluster_times5})
tree_learning_curve=dict({1:train_sizes6,2:train_scores6,3:test_scores6,4:fit_times6,5:score_times6,6:cluster_times6})
forrest_learning_curve=dict({1:train_sizes7,2:train_scores7,3:test_scores7,4:fit_times7,5:score_times7,6:cluster_times7})
adaboost_learning_curve=dict({1:train_sizes8,2:train_scores8,3:test_scores8,4:fit_times8,5:score_times8,6:cluster_times8})
gnb_learning_curve=dict({1:train_sizes9,2:train_scores9,3:test_scores9,4:fit_times9,5:score_times9,6:cluster_times9})
gpc_learning_curve=dict({1:train_sizes10,2:train_scores10,3:test_scores10,4:fit_times10,5:score_times10,6:cluster_times10})
ridge_learning_curve=dict({1:train_sizes11,2:train_scores11,3:test_scores11,4:fit_times11,5:score_times11,6:cluster_times11})

In [ ]:
names=["accuracy","macro_avg_precision","macro_avg_recall","macro_avg_f1_score",
       "micro_avg_precision","micro_avg_recall","micro_avg_f1_score",
       "weighted_avg_precision","weighted_avg_recall","weighted_avg_f1_score",
       "fit_time","cluster_time"]

df_svc = pd.DataFrame(index=names)
df_mlp = pd.DataFrame(index=names)
df_knn = pd.DataFrame(index=names)
df_logreg = pd.DataFrame(index=names)
df_linsvc = pd.DataFrame(index=names)
df_tree = pd.DataFrame(index=names)
df_forrest = pd.DataFrame(index=names)
df_adaboost = pd.DataFrame(index=names)
df_gnb = pd.DataFrame(index=names)
df_gpc = pd.DataFrame(index=names)
df_ridge = pd.DataFrame(index=names)

faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=train_path
    
    #kpts, des = orb.detectAndCompute(image_gray, None)
    kpts, des = brisk.detectAndCompute(face, None)
    #kpts, des = surf.detectAndCompute(im, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        print(image_path)
        list_to_delete.append(i)


new_faces_ = [j for i, j in enumerate(faces) if i not in list_to_delete]
faces=new_faces_

new_labels_ = [j for i, j in enumerate(labels) if i not in list_to_delete]
labels=new_labels_
print("done in %0.3fs" % (time() - t0))
print("Found descriptors for ",len(des_list)," images")
print("Found  ",len(labels)," labels")

labels=np.array(labels)
des_list=np.array(des_list)

In [ ]:
for size in range(1,6):
    print("--------------------------Start for size: ",size,"----------------------------------------")
    for k_f,idx in enumerate(kf.split(X=des_list, y=labels)):
        train_idx,test_idx=idx[0],idx[1]
        x_train=des_list[train_idx]
        y_train=labels[train_idx]

        x_test=des_list[test_idx]
        y_test=labels[test_idx]

        print("Fold Number : ",k_f+1)

        #Training==================================================================================================================================    
        if (size==1):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.22,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.22,shuffle=True,stratify=y_test)
        elif (size==2):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.325,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.325,shuffle=True,stratify=y_test)
        elif (size==3):
            X_train_new, X, y_train_new, y = train_test_split(x_train, y_train, train_size=0.55,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.55,shuffle=True,stratify=y_test)
        elif (size==4):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.77,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.77,shuffle=True,stratify=y_test)
        else:
            X_train_new=x_train
            y_train_new=y_train
            X_test_new=x_test
            y_test_new=y_test
        
        svc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        mlp_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        knn_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        logreg_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        linsvc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        tree_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        forrest_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        adaboost_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        gnb_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        if (k_f<5):
            gpc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        ridge_learning_curve[1][size-1]=len(X_train_new)#setting train_size

        print("Stacking...")
        t0 = time()
        descriptors = X_train_new[0][1]
        for image_path, descriptor in X_train_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        descriptors_float = descriptors.astype(float) 

        print("Creating clusters and histogram...")
        t0 = time()
        k = 200
        voc, variance = kmeans(descriptors_float, k, 1)

        im_features = np.zeros((len(X_train_new), k), "float32")
        for i in range(len(X_train_new)):
            words, distance = vq(X_train_new[i][1],voc)
            for w in words:
                im_features[i][w] += 1

        nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_train_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        stdSlr = StandardScaler().fit(im_features)
        im_features = stdSlr.transform(im_features)
        cluster_time=time() - t0
        print("done in %0.3fs" %cluster_time)
        svc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        mlp_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        knn_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        logreg_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        linsvc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        tree_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        forrest_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        adaboost_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        gnb_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        if (k_f<5):
            gpc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        ridge_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time


        #EDW MPAINEI TO MODELO GIA TO FITTING
        print("Training the svc model...")############################################################################
        t0 = time()

        svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
        svc.fit(im_features, np.array(y_train_new))

        svc_fit_time=time() - t0
        print("Training the svc done in %0.3fs" %svc_fit_time)
        svc_learning_curve[4][size-1][k_f]=svc_fit_time#setting fit_time
        test_sc=svc.score(im_features, np.array(y_train_new))
        svc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        
        print("Training the linsvc model...")###################################################################
        t0 = time()

        linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
        linsvc.fit(im_features, np.array(y_train_new))

        linsvc_fit_time=time() - t0
        print("Training the linsvc done in %0.3fs" %linsvc_fit_time)
        linsvc_learning_curve[4][size-1][k_f]=linsvc_fit_time#setting fit_time
        test_sc=linsvc.score(im_features, np.array(y_train_new))
        linsvc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the forrest model...")#####################################################################
        t0 = time()

        forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
        forrest.fit(im_features, np.array(y_train_new))

        forrest_fit_time=time() - t0
        print("Training the forrest done in %0.3fs" %forrest_fit_time)
        forrest_learning_curve[4][size-1][k_f]=forrest_fit_time#setting fit_time
        test_sc=forrest.score(im_features, np.array(y_train_new))
        forrest_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the logreg model...")#######################################################################
        t0 = time()

        logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
        logreg.fit(im_features, np.array(y_train_new))

        logreg_fit_time=time() - t0
        print("Training the logreg done in %0.3fs" %logreg_fit_time)
        logreg_learning_curve[4][size-1][k_f]=logreg_fit_time#setting fit_time
        test_sc=logreg.score(im_features, np.array(y_train_new))
        logreg_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the tree model...")#######################################################################
        t0 = time()

        tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                                  max_leaf_nodes=7,max_features=None)    
        tree.fit(im_features, np.array(y_train_new))

        tree_fit_time=time() - t0
        print("Training the tree done in %0.3fs" %tree_fit_time)
        tree_learning_curve[4][size-1][k_f]=tree_fit_time#setting fit_time
        test_sc=tree.score(im_features, np.array(y_train_new))
        tree_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the knn model...")#########################################################################
        t0 = time()

        knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
        knn.fit(im_features, np.array(y_train_new))

        knn_fit_time=time() - t0
        print("Training the knn done in %0.3fs" %knn_fit_time)
        knn_learning_curve[4][size-1][k_f]=knn_fit_time#setting fit_time
        test_sc=knn.score(im_features, np.array(y_train_new))
        knn_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the mlp model...")###########################################################################
        t0 = time()

        mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
        mlp.fit(im_features, np.array(y_train_new))

        mlp_fit_time=time() - t0
        print("Training the mlp done in %0.3fs" %mlp_fit_time)
        mlp_learning_curve[4][size-1][k_f]=mlp_fit_time#setting fit_time
        test_sc=mlp.score(im_features, np.array(y_train_new))
        mlp_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the adaboost model...")########################################################################
        t0 = time()

        adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
        adaboost.fit(im_features, np.array(y_train_new))

        adaboost_fit_time=time() - t0
        print("Training the adaboost done in %0.3fs" %adaboost_fit_time)
        adaboost_learning_curve[4][size-1][k_f]=adaboost_fit_time#setting fit_time
        test_sc=adaboost.score(im_features, np.array(y_train_new))
        adaboost_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gnb model...")###############################################################
        t0 = time()

        gnb = GaussianNB()    
        gnb.fit(im_features, np.array(y_train_new))

        gnb_fit_time=time() - t0
        print("Training the gnb done in %0.3fs" %gnb_fit_time)
        gnb_learning_curve[4][size-1][k_f]=gnb_fit_time#setting fit_time
        test_sc=gnb.score(im_features, np.array(y_train_new))
        gnb_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gpc model...")##############################################################
        if (k_f<5):
            t0 = time()

            gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
            gpc.fit(im_features, np.array(y_train_new))

            gpc_fit_time=time() - t0
            print("Training the gpc done in %0.3fs" %gpc_fit_time)
            gpc_learning_curve[4][size-1][k_f]=gpc_fit_time#setting fit_time
            test_sc=gpc.score(im_features, np.array(y_train_new))
            gpc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the ridge model...")############################################################
        t0 = time()

        ridge = RidgeClassifier(alpha=0.5,solver='saga')    
        ridge.fit(im_features, np.array(y_train_new))

        ridge_fit_time=time() - t0
        print("Training the ridge done in %0.3fs" %ridge_fit_time)
        ridge_learning_curve[4][size-1][k_f]=ridge_fit_time#setting fit_time
        test_sc=ridge.score(im_features, np.array(y_train_new))
        ridge_learning_curve[2][size-1][k_f]=test_sc#setting train_score



        #Testing=============================================================================================================================
        print("Testing...")     

        print("Stacking...")
        t0 = time()
        descriptors = X_test_new[0][1]
        for image_path, descriptor in X_test_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        print("Calculating histogram, Scaling...")
        test_features = np.zeros((len(X_test_new), k), "float32")
        for i in range(len(X_test_new)):
            words, distance = vq(X_test_new[i][1],voc)
            for w in words:
                test_features[i][w] += 1

        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_test_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)
        print("done in %0.3fs" % (time() - t0))

        if (size==5):
        
            true_class =  [training_names[i-1] for i in y_test_new]
            print("Predicting Started")
            #svc========================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in svc.predict(test_features)]
            score_t=time() - t0
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            svc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_svc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       svc_fit_time,cluster_time]

            #mlp====================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in mlp.predict(test_features)]
            score_t=time() - t0
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            mlp_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_mlp[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       mlp_fit_time,cluster_time]

            #knn=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in knn.predict(test_features)]
            score_t=time() - t0
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            knn_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_knn[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       knn_fit_time,cluster_time]

            #logreg==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in logreg.predict(test_features)]
            score_t=time() - t0
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            logreg_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_logreg[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       logreg_fit_time,cluster_time]

            #linsvc===================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in linsvc.predict(test_features)]
            score_t=time() - t0
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            linsvc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_linsvc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       linsvc_fit_time,cluster_time]

            #tree=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in tree.predict(test_features)]
            score_t=time() - t0
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            tree_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_tree[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       tree_fit_time,cluster_time]

            #forrest======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in forrest.predict(test_features)]
            score_t=time() - t0
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            forrest_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_forrest[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       forrest_fit_time,cluster_time]

            #adaboost=======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in adaboost.predict(test_features)]
            score_t=time() - t0
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            adaboost_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_adaboost[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       adaboost_fit_time,cluster_time]

            #gnb=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in gnb.predict(test_features)]
            score_t=time() - t0
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            gnb_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_gnb[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       gnb_fit_time,cluster_time]

            #gpc============================================================================================
            if (k_f<5):
                t0 = time()
                predictions =  [training_names[i-1] for i in gpc.predict(test_features)]
                score_t=time() - t0
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time


                report=classification_report(true_class, predictions,output_dict=True)
                accuracy=report['accuracy']
                gpc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
                macro_avg_precision=report['macro avg']['precision']
                macro_avg_recall=report['macro avg']['recall']
                macro_avg_f1_score=report['macro avg']['f1-score']
                micro_avg_precision=precision_score(true_class, predictions, average='micro')
                micro_avg_recall=recall_score(true_class, predictions, average='micro')
                micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
                weighted_avg_precision=report['weighted avg']['precision']
                weighted_avg_recall=report['weighted avg']['recall']
                weighted_avg_f1_score=report['weighted avg']['f1-score']

                df_gpc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                           micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                           weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                           gpc_fit_time,cluster_time]

            #ridge==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in ridge.predict(test_features)]
            score_t=time() - t0
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            ridge_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_ridge[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       ridge_fit_time,cluster_time]

        else:
            #svc=============================
            t0 = time()
            test_scores=svc.score(test_features,y_test_new)
            score_t=time() - t0
            svc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #mlp==================================
            t0 = time()
            test_scores=mlp.score(test_features,y_test_new)
            score_t=time() - t0
            mlp_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #knn=============================
            t0 = time()
            test_scores=knn.score(test_features,y_test_new)
            score_t=time() - t0
            knn_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #logreg====================================
            t0 = time()
            test_scores=logreg.score(test_features,y_test_new)
            score_t=time() - t0
            logreg_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #linsvc=========================================
            t0 = time()
            test_scores=linsvc.score(test_features,y_test_new)
            score_t=time() - t0
            linsvc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #tree======================================
            t0 = time()
            test_scores=tree.score(test_features,y_test_new)
            score_t=time() - t0
            tree_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #forrest=========================================
            t0 = time()
            test_scores=forrest.score(test_features,y_test_new)
            score_t=time() - t0
            forrest_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #adaboost==========================================
            t0 = time()
            test_scores=adaboost.score(test_features,y_test_new)
            score_t=time() - t0
            adaboost_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gnb===========================================
            t0 = time()
            test_scores=gnb.score(test_features,y_test_new)
            score_t=time() - t0
            gnb_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gpc========================================
            if (k_f<5):
                t0 = time()
                test_scores=gpc.score(test_features,y_test_new)
                score_t=time() - t0
                gpc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #ridge=========================================
            t0 = time()
            test_scores=ridge.score(test_features,y_test_new)
            score_t=time() - t0
            ridge_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            
        print("Predicting DONE for fold ",k_f+1)

        
    print("END OF LOOP for size: ",size)
    
print("End of ALL")
brisk['svc']=df_svc
brisk['mlp']=df_mlp
brisk['knn']=df_knn
brisk['logreg']=df_logreg
brisk['linsvc']=df_linsvc
brisk['tree']=df_tree
brisk['forrest']=df_forrest
brisk['adaboost']=df_adaboost
brisk['gnb']=df_gnb
brisk['gpc']=df_gpc
brisk['ridge']=df_ridge


brisk_learning_curve['svc']=svc_learning_curve
brisk_learning_curve['mlp']=mlp_learning_curve
brisk_learning_curve['knn']=knn_learning_curve
brisk_learning_curve['logreg']=logreg_learning_curve
brisk_learning_curve['linsvc']=linsvc_learning_curve
brisk_learning_curve['tree']=tree_learning_curve
brisk_learning_curve['forrest']=forrest_learning_curve
brisk_learning_curve['adaboost']=adaboost_learning_curve
brisk_learning_curve['gnb']=gnb_learning_curve
brisk_learning_curve['gpc']=gpc_learning_curve
brisk_learning_curvec['ridge']=ridge_learning_curve

In [ ]:
pickle_out = open('brisk', 'wb')
pickle.dump(brisk_lfw, pickle_out)
pickle_out.close()
#pickle_in = open('my_dict.pickle', 'rb')
#new_dict = pickle.load(pickle_in)

In [ ]:
with open('brisk_learning_curve.p', 'wb') as fp:
    pickle.dump(brisk_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

SIFT

In [ ]:
train_sizes1=np.empty(5)
train_sizes1[:]=np.NaN
train_scores1=np.empty([5, 10])
train_scores1[:]=np.NaN
test_scores1=np.empty([5, 10])
test_scores1[:]=np.NaN
fit_times1=np.empty([5, 10])
fit_times1[:]=np.NaN
score_times1=np.empty([5, 10])
score_times1[:]=np.NaN
cluster_times1=np.empty([5, 10])
cluster_times1[:]=np.NaN

train_sizes2=np.empty(5)
train_sizes2[:]=np.NaN
train_scores2=np.empty([5, 10])
train_scores2[:]=np.NaN
test_scores2=np.empty([5, 10])
test_scores2[:]=np.NaN
fit_times2=np.empty([5, 10])
fit_times2[:]=np.NaN
score_times2=np.empty([5, 10])
score_times2[:]=np.NaN
cluster_times2=np.empty([5, 10])
cluster_times2[:]=np.NaN

train_sizes3=np.empty(5)
train_sizes3[:]=np.NaN
train_scores3=np.empty([5, 10])
train_scores3[:]=np.NaN
test_scores3=np.empty([5, 10])
test_scores3[:]=np.NaN
fit_times3=np.empty([5, 10])
fit_times3[:]=np.NaN
score_times3=np.empty([5, 10])
score_times3[:]=np.NaN
cluster_times3=np.empty([5, 10])
cluster_times3[:]=np.NaN

train_sizes4=np.empty(5)
train_sizes4[:]=np.NaN
train_scores4=np.empty([5, 10])
train_scores4[:]=np.NaN
test_scores4=np.empty([5, 10])
test_scores4[:]=np.NaN
fit_times4=np.empty([5, 10])
fit_times4[:]=np.NaN
score_times4=np.empty([5, 10])
score_times4[:]=np.NaN
cluster_times4=np.empty([5, 10])
cluster_times4[:]=np.NaN

train_sizes5=np.empty(5)
train_sizes5[:]=np.NaN
train_scores5=np.empty([5, 10])
train_scores5[:]=np.NaN
test_scores5=np.empty([5, 10])
test_scores5[:]=np.NaN
fit_times5=np.empty([5, 10])
fit_times5[:]=np.NaN
score_times5=np.empty([5, 10])
score_times5[:]=np.NaN
cluster_times5=np.empty([5, 10])
cluster_times5[:]=np.NaN

train_sizes6=np.empty(5)
train_sizes6[:]=np.NaN
train_scores6=np.empty([5, 10])
train_scores6[:]=np.NaN
test_scores6=np.empty([5, 10])
test_scores6[:]=np.NaN
fit_times6=np.empty([5, 10])
fit_times6[:]=np.NaN
score_times6=np.empty([5, 10])
score_times6[:]=np.NaN
cluster_times6=np.empty([5, 10])
cluster_times6[:]=np.NaN

train_sizes7=np.empty(5)
train_sizes7[:]=np.NaN
train_scores7=np.empty([5, 10])
train_scores7[:]=np.NaN
test_scores7=np.empty([5, 10])
test_scores7[:]=np.NaN
fit_times7=np.empty([5, 10])
fit_times7[:]=np.NaN
score_times7=np.empty([5, 10])
score_times7[:]=np.NaN
cluster_times7=np.empty([5, 10])
cluster_times7[:]=np.NaN

train_sizes8=np.empty(5)
train_sizes8[:]=np.NaN
train_scores8=np.empty([5, 10])
train_scores8[:]=np.NaN
test_scores8=np.empty([5, 10])
test_scores8[:]=np.NaN
fit_times8=np.empty([5, 10])
fit_times8[:]=np.NaN
score_times8=np.empty([5, 10])
score_times8[:]=np.NaN
cluster_times8=np.empty([5, 10])
cluster_times8[:]=np.NaN

train_sizes9=np.empty(5)
train_sizes9[:]=np.NaN
train_scores9=np.empty([5, 10])
train_scores9[:]=np.NaN
test_scores9=np.empty([5, 10])
test_scores9[:]=np.NaN
fit_times9=np.empty([5, 10])
fit_times9[:]=np.NaN
score_times9=np.empty([5, 10])
score_times9[:]=np.NaN
cluster_times9=np.empty([5, 10])
cluster_times9[:]=np.NaN

train_sizes10=np.empty(5)
train_sizes10[:]=np.NaN
train_scores10=np.empty([5, 5])
train_scores10[:]=np.NaN
test_scores10=np.empty([5, 5])
test_scores10[:]=np.NaN
fit_times10=np.empty([5, 5])
fit_times10[:]=np.NaN
score_times10=np.empty([5, 5])
score_times10[:]=np.NaN
cluster_times10=np.empty([5, 5])
cluster_times10[:]=np.NaN

train_sizes11=np.empty(5)
train_sizes11[:]=np.NaN
train_scores11=np.empty([5, 10])
train_scores11[:]=np.NaN
test_scores11=np.empty([5, 10])
test_scores11[:]=np.NaN
fit_times11=np.empty([5, 10])
fit_times11[:]=np.NaN
score_times11=np.empty([5, 10])
score_times11[:]=np.NaN
cluster_times11=np.empty([5, 10])
cluster_times11[:]=np.NaN

In [ ]:
svc_learning_curve=dict({1:train_sizes1,2:train_scores1,3:test_scores1,4:fit_times1,5:score_times1,6:cluster_times1})
mlp_learning_curve=dict({1:train_sizes2,2:train_scores2,3:test_scores2,4:fit_times2,5:score_times2,6:cluster_times2})
knn_learning_curve=dict({1:train_sizes3,2:train_scores3,3:test_scores3,4:fit_times3,5:score_times3,6:cluster_times3})
logreg_learning_curve=dict({1:train_sizes4,2:train_scores4,3:test_scores4,4:fit_times4,5:score_times4,6:cluster_times4})
linsvc_learning_curve=dict({1:train_sizes5,2:train_scores5,3:test_scores5,4:fit_times5,5:score_times5,6:cluster_times5})
tree_learning_curve=dict({1:train_sizes6,2:train_scores6,3:test_scores6,4:fit_times6,5:score_times6,6:cluster_times6})
forrest_learning_curve=dict({1:train_sizes7,2:train_scores7,3:test_scores7,4:fit_times7,5:score_times7,6:cluster_times7})
adaboost_learning_curve=dict({1:train_sizes8,2:train_scores8,3:test_scores8,4:fit_times8,5:score_times8,6:cluster_times8})
gnb_learning_curve=dict({1:train_sizes9,2:train_scores9,3:test_scores9,4:fit_times9,5:score_times9,6:cluster_times9})
gpc_learning_curve=dict({1:train_sizes10,2:train_scores10,3:test_scores10,4:fit_times10,5:score_times10,6:cluster_times10})
ridge_learning_curve=dict({1:train_sizes11,2:train_scores11,3:test_scores11,4:fit_times11,5:score_times11,6:cluster_times11})

In [ ]:
names=["accuracy","macro_avg_precision","macro_avg_recall","macro_avg_f1_score",
       "micro_avg_precision","micro_avg_recall","micro_avg_f1_score",
       "weighted_avg_precision","weighted_avg_recall","weighted_avg_f1_score",
       "fit_time","cluster_time"]

df_svc = pd.DataFrame(index=names)
df_mlp = pd.DataFrame(index=names)
df_knn = pd.DataFrame(index=names)
df_logreg = pd.DataFrame(index=names)
df_linsvc = pd.DataFrame(index=names)
df_tree = pd.DataFrame(index=names)
df_forrest = pd.DataFrame(index=names)
df_adaboost = pd.DataFrame(index=names)
df_gnb = pd.DataFrame(index=names)
df_gpc = pd.DataFrame(index=names)
df_ridge = pd.DataFrame(index=names)

faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=train_path
    
    #kpts, des = orb.detectAndCompute(image_gray, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    #kpts, des = surf.detectAndCompute(im, None)
    kpts, des = sift.detectAndCompute(face, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        print(image_path)
        list_to_delete.append(i)


new_faces_ = [j for i, j in enumerate(faces) if i not in list_to_delete]
faces=new_faces_

new_labels_ = [j for i, j in enumerate(labels) if i not in list_to_delete]
labels=new_labels_
print("done in %0.3fs" % (time() - t0))
print("Found descriptors for ",len(des_list)," images")
print("Found  ",len(labels)," labels")

labels=np.array(labels)
des_list=np.array(des_list)

In [ ]:
for size in range(1,6):
    print("--------------------------Start for size: ",size,"----------------------------------------")
    for k_f,idx in enumerate(kf.split(X=des_list, y=labels)):
        train_idx,test_idx=idx[0],idx[1]
        x_train=des_list[train_idx]
        y_train=labels[train_idx]

        x_test=des_list[test_idx]
        y_test=labels[test_idx]

        print("Fold Number : ",k_f+1)

        #Training==================================================================================================================================    
        if (size==1):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.2,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.22,shuffle=True,stratify=y_test)
        elif (size==2):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.325,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.325,shuffle=True,stratify=y_test)
        elif (size==3):
            X_train_new, X, y_train_new, y = train_test_split(x_train, y_train, train_size=0.55,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.55,shuffle=True,stratify=y_test)
        elif (size==4):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.77,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.77,shuffle=True,stratify=y_test)
        else:
            X_train_new=x_train
            y_train_new=y_train
            X_test_new=x_test
            y_test_new=y_test
        
        svc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        mlp_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        knn_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        logreg_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        linsvc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        tree_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        forrest_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        adaboost_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        gnb_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        if (k_f<5):
            gpc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        ridge_learning_curve[1][size-1]=len(X_train_new)#setting train_size

        print("Stacking...")
        t0 = time()
        descriptors = X_train_new[0][1]
        for image_path, descriptor in X_train_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        descriptors_float = descriptors.astype(float) 

        print("Creating clusters and histogram...")
        t0 = time()
        k = 200
        voc, variance = kmeans(descriptors_float, k, 1)

        im_features = np.zeros((len(X_train_new), k), "float32")
        for i in range(len(X_train_new)):
            words, distance = vq(X_train_new[i][1],voc)
            for w in words:
                im_features[i][w] += 1

        nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_train_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        stdSlr = StandardScaler().fit(im_features)
        im_features = stdSlr.transform(im_features)
        cluster_time=time() - t0
        print("done in %0.3fs" %cluster_time)
        svc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        mlp_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        knn_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        logreg_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        linsvc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        tree_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        forrest_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        adaboost_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        gnb_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        if (k_f<5):
            gpc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        ridge_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time


        #EDW MPAINEI TO MODELO GIA TO FITTING
        print("Training the svc model...")############################################################################
        t0 = time()

        svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
        svc.fit(im_features, np.array(y_train_new))

        svc_fit_time=time() - t0
        print("Training the svc done in %0.3fs" %svc_fit_time)
        svc_learning_curve[4][size-1][k_f]=svc_fit_time#setting fit_time
        test_sc=svc.score(im_features, np.array(y_train_new))
        svc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        
        print("Training the linsvc model...")###################################################################
        t0 = time()

        linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
        linsvc.fit(im_features, np.array(y_train_new))

        linsvc_fit_time=time() - t0
        print("Training the linsvc done in %0.3fs" %linsvc_fit_time)
        linsvc_learning_curve[4][size-1][k_f]=linsvc_fit_time#setting fit_time
        test_sc=linsvc.score(im_features, np.array(y_train_new))
        linsvc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the forrest model...")#####################################################################
        t0 = time()

        forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
        forrest.fit(im_features, np.array(y_train_new))

        forrest_fit_time=time() - t0
        print("Training the forrest done in %0.3fs" %forrest_fit_time)
        forrest_learning_curve[4][size-1][k_f]=forrest_fit_time#setting fit_time
        test_sc=forrest.score(im_features, np.array(y_train_new))
        forrest_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the logreg model...")#######################################################################
        t0 = time()

        logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
        logreg.fit(im_features, np.array(y_train_new))

        logreg_fit_time=time() - t0
        print("Training the logreg done in %0.3fs" %logreg_fit_time)
        logreg_learning_curve[4][size-1][k_f]=logreg_fit_time#setting fit_time
        test_sc=logreg.score(im_features, np.array(y_train_new))
        logreg_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the tree model...")#######################################################################
        t0 = time()

        tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                                  max_leaf_nodes=7,max_features=None)    
        tree.fit(im_features, np.array(y_train_new))

        tree_fit_time=time() - t0
        print("Training the tree done in %0.3fs" %tree_fit_time)
        tree_learning_curve[4][size-1][k_f]=tree_fit_time#setting fit_time
        test_sc=tree.score(im_features, np.array(y_train_new))
        tree_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the knn model...")#########################################################################
        t0 = time()

        knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
        knn.fit(im_features, np.array(y_train_new))

        knn_fit_time=time() - t0
        print("Training the knn done in %0.3fs" %knn_fit_time)
        knn_learning_curve[4][size-1][k_f]=knn_fit_time#setting fit_time
        test_sc=knn.score(im_features, np.array(y_train_new))
        knn_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the mlp model...")###########################################################################
        t0 = time()

        mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
        mlp.fit(im_features, np.array(y_train_new))

        mlp_fit_time=time() - t0
        print("Training the mlp done in %0.3fs" %mlp_fit_time)
        mlp_learning_curve[4][size-1][k_f]=mlp_fit_time#setting fit_time
        test_sc=mlp.score(im_features, np.array(y_train_new))
        mlp_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the adaboost model...")########################################################################
        t0 = time()

        adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
        adaboost.fit(im_features, np.array(y_train_new))

        adaboost_fit_time=time() - t0
        print("Training the adaboost done in %0.3fs" %adaboost_fit_time)
        adaboost_learning_curve[4][size-1][k_f]=adaboost_fit_time#setting fit_time
        test_sc=adaboost.score(im_features, np.array(y_train_new))
        adaboost_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gnb model...")###############################################################
        t0 = time()

        gnb = GaussianNB()    
        gnb.fit(im_features, np.array(y_train_new))

        gnb_fit_time=time() - t0
        print("Training the gnb done in %0.3fs" %gnb_fit_time)
        gnb_learning_curve[4][size-1][k_f]=gnb_fit_time#setting fit_time
        test_sc=gnb.score(im_features, np.array(y_train_new))
        gnb_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gpc model...")##############################################################
        if (k_f<5):
            t0 = time()

            gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
            gpc.fit(im_features, np.array(y_train_new))

            gpc_fit_time=time() - t0
            print("Training the gpc done in %0.3fs" %gpc_fit_time)
            gpc_learning_curve[4][size-1][k_f]=gpc_fit_time#setting fit_time
            test_sc=gpc.score(im_features, np.array(y_train_new))
            gpc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the ridge model...")############################################################
        t0 = time()

        ridge = RidgeClassifier(alpha=0.5,solver='saga')    
        ridge.fit(im_features, np.array(y_train_new))

        ridge_fit_time=time() - t0
        print("Training the ridge done in %0.3fs" %ridge_fit_time)
        ridge_learning_curve[4][size-1][k_f]=ridge_fit_time#setting fit_time
        test_sc=ridge.score(im_features, np.array(y_train_new))
        ridge_learning_curve[2][size-1][k_f]=test_sc#setting train_score



        #Testing=============================================================================================================================
        print("Testing...")     

        print("Stacking...")
        t0 = time()
        descriptors = X_test_new[0][1]
        for image_path, descriptor in X_test_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        print("Calculating histogram, Scaling...")
        test_features = np.zeros((len(X_test_new), k), "float32")
        for i in range(len(X_test_new)):
            words, distance = vq(X_test_new[i][1],voc)
            for w in words:
                test_features[i][w] += 1

        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_test_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)
        print("done in %0.3fs" % (time() - t0))

        if (size==5):
        
            true_class =  [training_names[i-1] for i in y_test_new]
            print("Predicting Started")
            #svc========================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in svc.predict(test_features)]
            score_t=time() - t0
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            svc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_svc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       svc_fit_time,cluster_time]

            #mlp====================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in mlp.predict(test_features)]
            score_t=time() - t0
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            mlp_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_mlp[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       mlp_fit_time,cluster_time]

            #knn=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in knn.predict(test_features)]
            score_t=time() - t0
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            knn_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_knn[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       knn_fit_time,cluster_time]

            #logreg==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in logreg.predict(test_features)]
            score_t=time() - t0
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            logreg_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_logreg[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       logreg_fit_time,cluster_time]

            #linsvc===================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in linsvc.predict(test_features)]
            score_t=time() - t0
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            linsvc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_linsvc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       linsvc_fit_time,cluster_time]

            #tree=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in tree.predict(test_features)]
            score_t=time() - t0
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            tree_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_tree[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       tree_fit_time,cluster_time]

            #forrest======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in forrest.predict(test_features)]
            score_t=time() - t0
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            forrest_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_forrest[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       forrest_fit_time,cluster_time]

            #adaboost=======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in adaboost.predict(test_features)]
            score_t=time() - t0
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            adaboost_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_adaboost[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       adaboost_fit_time,cluster_time]

            #gnb=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in gnb.predict(test_features)]
            score_t=time() - t0
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            gnb_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_gnb[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       gnb_fit_time,cluster_time]

            #gpc============================================================================================
            if (k_f<5):
                t0 = time()
                predictions =  [training_names[i-1] for i in gpc.predict(test_features)]
                score_t=time() - t0
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time


                report=classification_report(true_class, predictions,output_dict=True)
                accuracy=report['accuracy']
                gpc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
                macro_avg_precision=report['macro avg']['precision']
                macro_avg_recall=report['macro avg']['recall']
                macro_avg_f1_score=report['macro avg']['f1-score']
                micro_avg_precision=precision_score(true_class, predictions, average='micro')
                micro_avg_recall=recall_score(true_class, predictions, average='micro')
                micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
                weighted_avg_precision=report['weighted avg']['precision']
                weighted_avg_recall=report['weighted avg']['recall']
                weighted_avg_f1_score=report['weighted avg']['f1-score']

                df_gpc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                           micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                           weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                           gpc_fit_time,cluster_time]

            #ridge==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in ridge.predict(test_features)]
            score_t=time() - t0
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            ridge_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_ridge[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       ridge_fit_time,cluster_time]

        else:
            #svc=============================
            t0 = time()
            test_scores=svc.score(test_features,y_test_new)
            score_t=time() - t0
            svc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #mlp==================================
            t0 = time()
            test_scores=mlp.score(test_features,y_test_new)
            score_t=time() - t0
            mlp_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #knn=============================
            t0 = time()
            test_scores=knn.score(test_features,y_test_new)
            score_t=time() - t0
            knn_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #logreg====================================
            t0 = time()
            test_scores=logreg.score(test_features,y_test_new)
            score_t=time() - t0
            logreg_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #linsvc=========================================
            t0 = time()
            test_scores=linsvc.score(test_features,y_test_new)
            score_t=time() - t0
            linsvc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #tree======================================
            t0 = time()
            test_scores=tree.score(test_features,y_test_new)
            score_t=time() - t0
            tree_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #forrest=========================================
            t0 = time()
            test_scores=forrest.score(test_features,y_test_new)
            score_t=time() - t0
            forrest_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #adaboost==========================================
            t0 = time()
            test_scores=adaboost.score(test_features,y_test_new)
            score_t=time() - t0
            adaboost_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gnb===========================================
            t0 = time()
            test_scores=gnb.score(test_features,y_test_new)
            score_t=time() - t0
            gnb_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gpc========================================
            if (k_f<5):
                t0 = time()
                test_scores=gpc.score(test_features,y_test_new)
                score_t=time() - t0
                gpc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #ridge=========================================
            t0 = time()
            test_scores=ridge.score(test_features,y_test_new)
            score_t=time() - t0
            ridge_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            
        print("Predicting DONE for fold ",k_f+1)

        
    print("END OF LOOP for size: ",size)
    
print("End of ALL")
sift['svc']=df_svc
sift['mlp']=df_mlp
sift['knn']=df_knn
sift['logreg']=df_logreg
sift['linsvc']=df_linsvc
sift['tree']=df_tree
sift['forrest']=df_forrest
sift['adaboost']=df_adaboost
sift['gnb']=df_gnb
sift['gpc']=df_gpc
sift['ridge']=df_ridge


sift_learning_curve['svc']=svc_learning_curve
sift_learning_curve['mlp']=mlp_learning_curve
sift_learning_curve['knn']=knn_learning_curve
sift_learning_curve['logreg']=logreg_learning_curve
sift_learning_curve['linsvc']=linsvc_learning_curve
sift_learning_curve['tree']=tree_learning_curve
sift_learning_curve['forrest']=forrest_learning_curve
sift_learning_curve['adaboost']=adaboost_learning_curve
sift_learning_curve['gnb']=gnb_learning_curve
sift_learning_curve['gpc']=gpc_learning_curve
sift_learning_curve['ridge']=ridge_learning_curve

In [ ]:
pickle_out = open('sift_extyale', 'wb')
pickle.dump(sift_lfw, pickle_out)
pickle_out.close()
#pickle_in = open('my_dict.pickle', 'rb')
#new_dict = pickle.load(pickle_in)

In [ ]:
with open('sift_learning_curve.p', 'wb') as fp:
    pickle.dump(sift_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

SURF

In [ ]:
train_sizes1=np.empty(5)
train_sizes1[:]=np.NaN
train_scores1=np.empty([5, 10])
train_scores1[:]=np.NaN
test_scores1=np.empty([5, 10])
test_scores1[:]=np.NaN
fit_times1=np.empty([5, 10])
fit_times1[:]=np.NaN
score_times1=np.empty([5, 10])
score_times1[:]=np.NaN
cluster_times1=np.empty([5, 10])
cluster_times1[:]=np.NaN

train_sizes2=np.empty(5)
train_sizes2[:]=np.NaN
train_scores2=np.empty([5, 10])
train_scores2[:]=np.NaN
test_scores2=np.empty([5, 10])
test_scores2[:]=np.NaN
fit_times2=np.empty([5, 10])
fit_times2[:]=np.NaN
score_times2=np.empty([5, 10])
score_times2[:]=np.NaN
cluster_times2=np.empty([5, 10])
cluster_times2[:]=np.NaN

train_sizes3=np.empty(5)
train_sizes3[:]=np.NaN
train_scores3=np.empty([5, 10])
train_scores3[:]=np.NaN
test_scores3=np.empty([5, 10])
test_scores3[:]=np.NaN
fit_times3=np.empty([5, 10])
fit_times3[:]=np.NaN
score_times3=np.empty([5, 10])
score_times3[:]=np.NaN
cluster_times3=np.empty([5, 10])
cluster_times3[:]=np.NaN

train_sizes4=np.empty(5)
train_sizes4[:]=np.NaN
train_scores4=np.empty([5, 10])
train_scores4[:]=np.NaN
test_scores4=np.empty([5, 10])
test_scores4[:]=np.NaN
fit_times4=np.empty([5, 10])
fit_times4[:]=np.NaN
score_times4=np.empty([5, 10])
score_times4[:]=np.NaN
cluster_times4=np.empty([5, 10])
cluster_times4[:]=np.NaN

train_sizes5=np.empty(5)
train_sizes5[:]=np.NaN
train_scores5=np.empty([5, 10])
train_scores5[:]=np.NaN
test_scores5=np.empty([5, 10])
test_scores5[:]=np.NaN
fit_times5=np.empty([5, 10])
fit_times5[:]=np.NaN
score_times5=np.empty([5, 10])
score_times5[:]=np.NaN
cluster_times5=np.empty([5, 10])
cluster_times5[:]=np.NaN

train_sizes6=np.empty(5)
train_sizes6[:]=np.NaN
train_scores6=np.empty([5, 10])
train_scores6[:]=np.NaN
test_scores6=np.empty([5, 10])
test_scores6[:]=np.NaN
fit_times6=np.empty([5, 10])
fit_times6[:]=np.NaN
score_times6=np.empty([5, 10])
score_times6[:]=np.NaN
cluster_times6=np.empty([5, 10])
cluster_times6[:]=np.NaN

train_sizes7=np.empty(5)
train_sizes7[:]=np.NaN
train_scores7=np.empty([5, 10])
train_scores7[:]=np.NaN
test_scores7=np.empty([5, 10])
test_scores7[:]=np.NaN
fit_times7=np.empty([5, 10])
fit_times7[:]=np.NaN
score_times7=np.empty([5, 10])
score_times7[:]=np.NaN
cluster_times7=np.empty([5, 10])
cluster_times7[:]=np.NaN

train_sizes8=np.empty(5)
train_sizes8[:]=np.NaN
train_scores8=np.empty([5, 10])
train_scores8[:]=np.NaN
test_scores8=np.empty([5, 10])
test_scores8[:]=np.NaN
fit_times8=np.empty([5, 10])
fit_times8[:]=np.NaN
score_times8=np.empty([5, 10])
score_times8[:]=np.NaN
cluster_times8=np.empty([5, 10])
cluster_times8[:]=np.NaN

train_sizes9=np.empty(5)
train_sizes9[:]=np.NaN
train_scores9=np.empty([5, 10])
train_scores9[:]=np.NaN
test_scores9=np.empty([5, 10])
test_scores9[:]=np.NaN
fit_times9=np.empty([5, 10])
fit_times9[:]=np.NaN
score_times9=np.empty([5, 10])
score_times9[:]=np.NaN
cluster_times9=np.empty([5, 10])
cluster_times9[:]=np.NaN

train_sizes10=np.empty(5)
train_sizes10[:]=np.NaN
train_scores10=np.empty([5, 5])
train_scores10[:]=np.NaN
test_scores10=np.empty([5, 5])
test_scores10[:]=np.NaN
fit_times10=np.empty([5, 5])
fit_times10[:]=np.NaN
score_times10=np.empty([5, 5])
score_times10[:]=np.NaN
cluster_times10=np.empty([5, 5])
cluster_times10[:]=np.NaN

train_sizes11=np.empty(5)
train_sizes11[:]=np.NaN
train_scores11=np.empty([5, 10])
train_scores11[:]=np.NaN
test_scores11=np.empty([5, 10])
test_scores11[:]=np.NaN
fit_times11=np.empty([5, 10])
fit_times11[:]=np.NaN
score_times11=np.empty([5, 10])
score_times11[:]=np.NaN
cluster_times11=np.empty([5, 10])
cluster_times11[:]=np.NaN

In [ ]:
svc_learning_curve=dict({1:train_sizes1,2:train_scores1,3:test_scores1,4:fit_times1,5:score_times1,6:cluster_times1})
mlp_learning_curve=dict({1:train_sizes2,2:train_scores2,3:test_scores2,4:fit_times2,5:score_times2,6:cluster_times2})
knn_learning_curve=dict({1:train_sizes3,2:train_scores3,3:test_scores3,4:fit_times3,5:score_times3,6:cluster_times3})
logreg_learning_curve=dict({1:train_sizes4,2:train_scores4,3:test_scores4,4:fit_times4,5:score_times4,6:cluster_times4})
linsvc_learning_curve=dict({1:train_sizes5,2:train_scores5,3:test_scores5,4:fit_times5,5:score_times5,6:cluster_times5})
tree_learning_curve=dict({1:train_sizes6,2:train_scores6,3:test_scores6,4:fit_times6,5:score_times6,6:cluster_times6})
forrest_learning_curve=dict({1:train_sizes7,2:train_scores7,3:test_scores7,4:fit_times7,5:score_times7,6:cluster_times7})
adaboost_learning_curve=dict({1:train_sizes8,2:train_scores8,3:test_scores8,4:fit_times8,5:score_times8,6:cluster_times8})
gnb_learning_curve=dict({1:train_sizes9,2:train_scores9,3:test_scores9,4:fit_times9,5:score_times9,6:cluster_times9})
gpc_learning_curve=dict({1:train_sizes10,2:train_scores10,3:test_scores10,4:fit_times10,5:score_times10,6:cluster_times10})
ridge_learning_curve=dict({1:train_sizes11,2:train_scores11,3:test_scores11,4:fit_times11,5:score_times11,6:cluster_times11})

In [ ]:
names=["accuracy","macro_avg_precision","macro_avg_recall","macro_avg_f1_score",
       "micro_avg_precision","micro_avg_recall","micro_avg_f1_score",
       "weighted_avg_precision","weighted_avg_recall","weighted_avg_f1_score",
       "fit_time","cluster_time"]

df_svc = pd.DataFrame(index=names)
df_mlp = pd.DataFrame(index=names)
df_knn = pd.DataFrame(index=names)
df_logreg = pd.DataFrame(index=names)
df_linsvc = pd.DataFrame(index=names)
df_tree = pd.DataFrame(index=names)
df_forrest = pd.DataFrame(index=names)
df_adaboost = pd.DataFrame(index=names)
df_gnb = pd.DataFrame(index=names)
df_gpc = pd.DataFrame(index=names)
df_ridge = pd.DataFrame(index=names)

faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=train_path
        
    #kpts, des = orb.detectAndCompute(image_gray, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    kpts, des = surf.detectAndCompute(face, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        print(image_path)
        list_to_delete.append(i)


new_faces_ = [j for i, j in enumerate(faces) if i not in list_to_delete]
faces=new_faces_

new_labels_ = [j for i, j in enumerate(labels) if i not in list_to_delete]
labels=new_labels_
print("done in %0.3fs" % (time() - t0))
print("Found descriptors for ",len(des_list)," images")
print("Found  ",len(labels)," labels")

labels=np.array(labels)
des_list=np.array(des_list)

In [ ]:
for size in range(1,6):
    print("--------------------------Start for size: ",size,"----------------------------------------")
    for k_f,idx in enumerate(kf.split(X=des_list, y=labels)):
        train_idx,test_idx=idx[0],idx[1]
        x_train=des_list[train_idx]
        y_train=labels[train_idx]

        x_test=des_list[test_idx]
        y_test=labels[test_idx]

        print("Fold Number : ",k_f+1)

        #Training==================================================================================================================================    
        if (size==1):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.22,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.22,shuffle=True,stratify=y_test)
        elif (size==2):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.325,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.325,shuffle=True,stratify=y_test)
        elif (size==3):
            X_train_new, X, y_train_new, y = train_test_split(x_train, y_train, train_size=0.55,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.55,shuffle=True,stratify=y_test)
        elif (size==4):
            X_train_new, X, y_train_new, y_ = train_test_split(x_train, y_train, train_size=0.77,shuffle=True,stratify=y_train)
            X_, X_test_new, y_, y_test_new = train_test_split(x_test, y_test, test_size=0.77,shuffle=True,stratify=y_test)
        else:
            X_train_new=x_train
            y_train_new=y_train
            X_test_new=x_test
            y_test_new=y_test
        
        svc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        mlp_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        knn_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        logreg_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        linsvc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        tree_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        forrest_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        adaboost_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        gnb_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        if (k_f<5):
            gpc_learning_curve[1][size-1]=len(X_train_new)#setting train_size
        ridge_learning_curve[1][size-1]=len(X_train_new)#setting train_size

        print("Stacking...")
        t0 = time()
        descriptors = X_train_new[0][1]
        for image_path, descriptor in X_train_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        descriptors_float = descriptors.astype(float) 

        print("Creating clusters and histogram...")
        t0 = time()
        k = 200
        voc, variance = kmeans(descriptors_float, k, 1)

        im_features = np.zeros((len(X_train_new), k), "float32")
        for i in range(len(X_train_new)):
            words, distance = vq(X_train_new[i][1],voc)
            for w in words:
                im_features[i][w] += 1

        nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_train_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        stdSlr = StandardScaler().fit(im_features)
        im_features = stdSlr.transform(im_features)
        cluster_time=time() - t0
        print("done in %0.3fs" %cluster_time)
        svc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        mlp_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        knn_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        logreg_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        linsvc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        tree_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        forrest_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        adaboost_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        gnb_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        if (k_f<5):
            gpc_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time
        ridge_learning_curve[6][size-1][k_f]=cluster_time#setting cluster_time


        #EDW MPAINEI TO MODELO GIA TO FITTING
        print("Training the svc model...")############################################################################
        t0 = time()

        svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
        svc.fit(im_features, np.array(y_train_new))

        svc_fit_time=time() - t0
        print("Training the svc done in %0.3fs" %svc_fit_time)
        svc_learning_curve[4][size-1][k_f]=svc_fit_time#setting fit_time
        test_sc=svc.score(im_features, np.array(y_train_new))
        svc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        
        print("Training the linsvc model...")###################################################################
        t0 = time()

        linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
        linsvc.fit(im_features, np.array(y_train_new))

        linsvc_fit_time=time() - t0
        print("Training the linsvc done in %0.3fs" %linsvc_fit_time)
        linsvc_learning_curve[4][size-1][k_f]=linsvc_fit_time#setting fit_time
        test_sc=linsvc.score(im_features, np.array(y_train_new))
        linsvc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the forrest model...")#####################################################################
        t0 = time()

        forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
        forrest.fit(im_features, np.array(y_train_new))

        forrest_fit_time=time() - t0
        print("Training the forrest done in %0.3fs" %forrest_fit_time)
        forrest_learning_curve[4][size-1][k_f]=forrest_fit_time#setting fit_time
        test_sc=forrest.score(im_features, np.array(y_train_new))
        forrest_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the logreg model...")#######################################################################
        t0 = time()

        logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
        logreg.fit(im_features, np.array(y_train_new))

        logreg_fit_time=time() - t0
        print("Training the logreg done in %0.3fs" %logreg_fit_time)
        logreg_learning_curve[4][size-1][k_f]=logreg_fit_time#setting fit_time
        test_sc=logreg.score(im_features, np.array(y_train_new))
        logreg_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the tree model...")#######################################################################
        t0 = time()

        tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                                  max_leaf_nodes=7,max_features=None)    
        tree.fit(im_features, np.array(y_train_new))

        tree_fit_time=time() - t0
        print("Training the tree done in %0.3fs" %tree_fit_time)
        tree_learning_curve[4][size-1][k_f]=tree_fit_time#setting fit_time
        test_sc=tree.score(im_features, np.array(y_train_new))
        tree_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the knn model...")#########################################################################
        t0 = time()

        knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
        knn.fit(im_features, np.array(y_train_new))

        knn_fit_time=time() - t0
        print("Training the knn done in %0.3fs" %knn_fit_time)
        knn_learning_curve[4][size-1][k_f]=knn_fit_time#setting fit_time
        test_sc=knn.score(im_features, np.array(y_train_new))
        knn_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the mlp model...")###########################################################################
        t0 = time()

        mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
        mlp.fit(im_features, np.array(y_train_new))

        mlp_fit_time=time() - t0
        print("Training the mlp done in %0.3fs" %mlp_fit_time)
        mlp_learning_curve[4][size-1][k_f]=mlp_fit_time#setting fit_time
        test_sc=mlp.score(im_features, np.array(y_train_new))
        mlp_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the adaboost model...")########################################################################
        t0 = time()

        adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
        adaboost.fit(im_features, np.array(y_train_new))

        adaboost_fit_time=time() - t0
        print("Training the adaboost done in %0.3fs" %adaboost_fit_time)
        adaboost_learning_curve[4][size-1][k_f]=adaboost_fit_time#setting fit_time
        test_sc=adaboost.score(im_features, np.array(y_train_new))
        adaboost_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gnb model...")###############################################################
        t0 = time()

        gnb = GaussianNB()    
        gnb.fit(im_features, np.array(y_train_new))

        gnb_fit_time=time() - t0
        print("Training the gnb done in %0.3fs" %gnb_fit_time)
        gnb_learning_curve[4][size-1][k_f]=gnb_fit_time#setting fit_time
        test_sc=gnb.score(im_features, np.array(y_train_new))
        gnb_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the gpc model...")##############################################################
        if (k_f<5):
            t0 = time()

            gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
            gpc.fit(im_features, np.array(y_train_new))

            gpc_fit_time=time() - t0
            print("Training the gpc done in %0.3fs" %gpc_fit_time)
            gpc_learning_curve[4][size-1][k_f]=gpc_fit_time#setting fit_time
            test_sc=gpc.score(im_features, np.array(y_train_new))
            gpc_learning_curve[2][size-1][k_f]=test_sc#setting train_score
        

        print("Training the ridge model...")############################################################
        t0 = time()

        ridge = RidgeClassifier(alpha=0.5,solver='saga')    
        ridge.fit(im_features, np.array(y_train_new))

        ridge_fit_time=time() - t0
        print("Training the ridge done in %0.3fs" %ridge_fit_time)
        ridge_learning_curve[4][size-1][k_f]=ridge_fit_time#setting fit_time
        test_sc=ridge.score(im_features, np.array(y_train_new))
        ridge_learning_curve[2][size-1][k_f]=test_sc#setting train_score



        #Testing=============================================================================================================================
        print("Testing...")     

        print("Stacking...")
        t0 = time()
        descriptors = X_test_new[0][1]
        for image_path, descriptor in X_test_new[1:]:
            descriptors = np.vstack((descriptors, descriptor))
        print("done in %0.3fs" % (time() - t0))

        print("Calculating histogram, Scaling...")
        test_features = np.zeros((len(X_test_new), k), "float32")
        for i in range(len(X_test_new)):
            words, distance = vq(X_test_new[i][1],voc)
            for w in words:
                test_features[i][w] += 1

        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*len(X_test_new)+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)
        print("done in %0.3fs" % (time() - t0))

        if (size==5):
        
            true_class =  [training_names[i-1] for i in y_test_new]
            print("Predicting Started")
            #svc========================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in svc.predict(test_features)]
            score_t=time() - t0
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time            

            report=classification_report(true_class, predictions, output_dict=True)
            accuracy=report['accuracy']
            svc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_svc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       svc_fit_time,cluster_time]

            #mlp====================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in mlp.predict(test_features)]
            score_t=time() - t0
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            mlp_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_mlp[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       mlp_fit_time,cluster_time]

            #knn=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in knn.predict(test_features)]
            score_t=time() - t0
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            knn_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_knn[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       knn_fit_time,cluster_time]

            #logreg==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in logreg.predict(test_features)]
            score_t=time() - t0
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            logreg_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_logreg[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       logreg_fit_time,cluster_time]

            #linsvc===================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in linsvc.predict(test_features)]
            score_t=time() - t0
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            linsvc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_linsvc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       linsvc_fit_time,cluster_time]

            #tree=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in tree.predict(test_features)]
            score_t=time() - t0
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            tree_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_tree[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       tree_fit_time,cluster_time]

            #forrest======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in forrest.predict(test_features)]
            score_t=time() - t0
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            forrest_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_forrest[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       forrest_fit_time,cluster_time]

            #adaboost=======================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in adaboost.predict(test_features)]
            score_t=time() - t0
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            adaboost_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_adaboost[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       adaboost_fit_time,cluster_time]

            #gnb=================================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in gnb.predict(test_features)]
            score_t=time() - t0
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            gnb_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_gnb[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       gnb_fit_time,cluster_time]

            #gpc============================================================================================
            if (k_f<5):
                t0 = time()
                predictions =  [training_names[i-1] for i in gpc.predict(test_features)]
                score_t=time() - t0
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time


                report=classification_report(true_class, predictions,output_dict=True)
                accuracy=report['accuracy']
                gpc_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
                macro_avg_precision=report['macro avg']['precision']
                macro_avg_recall=report['macro avg']['recall']
                macro_avg_f1_score=report['macro avg']['f1-score']
                micro_avg_precision=precision_score(true_class, predictions, average='micro')
                micro_avg_recall=recall_score(true_class, predictions, average='micro')
                micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
                weighted_avg_precision=report['weighted avg']['precision']
                weighted_avg_recall=report['weighted avg']['recall']
                weighted_avg_f1_score=report['weighted avg']['f1-score']

                df_gpc[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                           micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                           weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                           gpc_fit_time,cluster_time]

            #ridge==============================================================================================
            t0 = time()
            predictions =  [training_names[i-1] for i in ridge.predict(test_features)]
            score_t=time() - t0
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            

            report=classification_report(true_class, predictions,output_dict=True)
            accuracy=report['accuracy']
            ridge_learning_curve[3][size-1][k_f]=accuracy#setting test_scores
            macro_avg_precision=report['macro avg']['precision']
            macro_avg_recall=report['macro avg']['recall']
            macro_avg_f1_score=report['macro avg']['f1-score']
            micro_avg_precision=precision_score(true_class, predictions, average='micro')
            micro_avg_recall=recall_score(true_class, predictions, average='micro')
            micro_avg_f1_score=f1_score(true_class, predictions, average='micro')
            weighted_avg_precision=report['weighted avg']['precision']
            weighted_avg_recall=report['weighted avg']['recall']
            weighted_avg_f1_score=report['weighted avg']['f1-score']

            df_ridge[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
                       micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
                       weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                       ridge_fit_time,cluster_time]

        else:
            #svc=============================
            t0 = time()
            test_scores=svc.score(test_features,y_test_new)
            score_t=time() - t0
            svc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            svc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #mlp==================================
            t0 = time()
            test_scores=mlp.score(test_features,y_test_new)
            score_t=time() - t0
            mlp_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            mlp_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #knn=============================
            t0 = time()
            test_scores=knn.score(test_features,y_test_new)
            score_t=time() - t0
            knn_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            knn_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #logreg====================================
            t0 = time()
            test_scores=logreg.score(test_features,y_test_new)
            score_t=time() - t0
            logreg_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            logreg_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #linsvc=========================================
            t0 = time()
            test_scores=linsvc.score(test_features,y_test_new)
            score_t=time() - t0
            linsvc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            linsvc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #tree======================================
            t0 = time()
            test_scores=tree.score(test_features,y_test_new)
            score_t=time() - t0
            tree_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            tree_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            #forrest=========================================
            t0 = time()
            test_scores=forrest.score(test_features,y_test_new)
            score_t=time() - t0
            forrest_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            forrest_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #adaboost==========================================
            t0 = time()
            test_scores=adaboost.score(test_features,y_test_new)
            score_t=time() - t0
            adaboost_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            adaboost_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gnb===========================================
            t0 = time()
            test_scores=gnb.score(test_features,y_test_new)
            score_t=time() - t0
            gnb_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            gnb_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #gpc========================================
            if (k_f<5):
                t0 = time()
                test_scores=gpc.score(test_features,y_test_new)
                score_t=time() - t0
                gpc_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
                gpc_learning_curve[5][size-1][k_f]=score_t#setting score_time

            #ridge=========================================
            t0 = time()
            test_scores=ridge.score(test_features,y_test_new)
            score_t=time() - t0
            ridge_learning_curve[3][size-1][k_f]=test_scores#setting test_scores
            ridge_learning_curve[5][size-1][k_f]=score_t#setting score_time
            
            
        print("Predicting DONE for fold ",k_f+1)

        
    print("END OF LOOP for size: ",size)
    
print("End of ALL")
surf['svc']=df_svc
surf['mlp']=df_mlp
surf['knn']=df_knn
surf['logreg']=df_logreg
surf['linsvc']=df_linsvc
surf['tree']=df_tree
surf['forrest']=df_forrest
surf['adaboost']=df_adaboost
surf['gnb']=df_gnb
surf['gpc']=df_gpc
surf['ridge']=df_ridge


surf_learning_curve['svc']=svc_learning_curve
surf_learning_curve['mlp']=mlp_learning_curve
surf_learning_curve['knn']=knn_learning_curve
surf_learning_curve['logreg']=logreg_learning_curve
surf_learning_curve['linsvc']=linsvc_learning_curve
surf_learning_curve['tree']=tree_learning_curve
surf_learning_curve['forrest']=forrest_learning_curve
surf_learning_curve['adaboost']=adaboost_learning_curve
surf_learning_curve['gnb']=gnb_learning_curve
surf_learning_curve['gpc']=gpc_learning_curve
surf_learning_curve['ridge']=ridge_learning_curve

In [ ]:
pickle_out = open('surf_extyale', 'wb')
pickle.dump(surf, pickle_out)
pickle_out.close()
#pickle_in = open('my_dict.pickle', 'rb')
#new_dict = pickle.load(pickle_in)

In [ ]:
with open('surf_learning_curve.p', 'wb') as fp:
    pickle.dump(surf_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)